In [43]:
import os
import pandas as pd
import urllib.parse
import qrcode
import re
print ("Biblioteca importada com sucesso!")

Biblioteca importada com sucesso!


In [44]:
#Definir constantes para caminhos e URLs

URL_BASE_QR = "https://datacanuto.github.io/galeria-digital-artes"

IMG_PATH = r"C:\Users\pedro\Documents\qrCodeTelasExposicao\assets\img"
DIRETORIO_BASE = os.getcwd()
DIRETORIO_SAIDA = os.path.join(DIRETORIO_BASE, "qrcodes")
QR_EXPORT_DIR = os.path.join(DIRETORIO_BASE, "qr_codes_export")
DADOS = os.path.join(DIRETORIO_BASE, "dados_obras.csv")

# Criar diretório para exportação de QR codes
os.makedirs(QR_EXPORT_DIR, exist_ok=True)
print(f"Diretório de exportação de QR codes: {QR_EXPORT_DIR}")

Diretório de exportação de QR codes: c:\Users\pedro\Documents\qrCodeTelasExposicao\qr_codes_export


In [45]:
df = pd.read_csv(DADOS)
df = pd.DataFrame(df)
print("Dados carregados com sucesso!")
print("Total de registros:", len(df))
df.head()

Dados carregados com sucesso!
Total de registros: 62


,ITEM,TELAS,TÉCNICA,DIMENSÃO,ANO,VALOR (R$),VALOR_PIX,PARCELAMENTO,CATEGORIA
0,1,PORTAS E JANELAS 1,ACRÍLICA SOBRE TELA,100 X 60,2018,"6.000,00","5.700,00",5% À VISTA/ 6 VEZES,TELAS
1,2,PORTAS E JANELAS 2,ACRÍLICA SOBRE TELA,70 X 120,2024,"8.400,00","7.980,00",5% À VISTA/ 6 VEZES,TELAS
2,3,PORTAS E JANELAS 3,ACRÍLICA SOBRE TELA,53 X 73,2024,"3.860,00","3.667,00",5% À VISTA/ 6 VEZES,TELAS
3,4,PORTAS E JANELAS 4,ACRÍLICA SOBRE TELA,50 X 70,2024,"3.500,00","3.325,00",5% À VISTA/ 6 VEZES,TELAS
4,5,PORTAS E JANELAS 5,ACRÍLICA SOBRE TELA,70 X 100,2024,"7.000,00","6.650,00",5% À VISTA/ 6 VEZES,TELAS


In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62 entries, 0 to 61
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ITEM          62 non-null     int64 
 1   TELAS         62 non-null     object
 2   TÉCNICA       62 non-null     object
 3   DIMENSÃO      62 non-null     object
 4   ANO           62 non-null     int64 
 5   VALOR (R$)    62 non-null     object
 6   VALOR_PIX     62 non-null     object
 7   PARCELAMENTO  62 non-null     object
 8   CATEGORIA     62 non-null     object
dtypes: int64(2), object(7)
memory usage: 4.5+ KB


In [47]:
df.columns = df.columns.str.strip()
df.columns = df.columns.str.lower()

print(df.columns)

Index(['item', 'telas', 'técnica', 'dimensão', 'ano', 'valor (r$)',
       'valor_pix', 'parcelamento', 'categoria'],
      dtype='object')


In [48]:
img_list = os.listdir(IMG_PATH)
img_fullpath = [os.path.join(IMG_PATH, img) for img in img_list]

# Criar dicionário para mapear índice com imagem
img_dict = {i+1: img for i, img in enumerate(img_list)}
df['fotos'] = df['item'].map(img_dict)

In [49]:
import numpy as np
df['status'] = np.where(df['valor (r$)'] != 0, 'disponível', 'acervo pessoal')
df['link_pix'] = 'link_pix'
df['link_mp'] = 'link_mp'
df['data_hora'] = 'data_hora_venda'
df['tipo_transacao'] = 'tipo_transacao'
df.head()

,item,telas,técnica,dimensão,ano,valor (r$),valor_pix,parcelamento,categoria,fotos,status,link_pix,link_mp,data_hora,tipo_transacao
0,1,PORTAS E JANELAS 1,ACRÍLICA SOBRE TELA,100 X 60,2018,"6.000,00","5.700,00",5% À VISTA/ 6 VEZES,TELAS,REF 01...PORTAS E JANELAS 1.jpg,disponível,link_pix,link_mp,data_hora_venda,tipo_transacao
1,2,PORTAS E JANELAS 2,ACRÍLICA SOBRE TELA,70 X 120,2024,"8.400,00","7.980,00",5% À VISTA/ 6 VEZES,TELAS,REF 02...PORTAS E JANELAS 2.jpg,disponível,link_pix,link_mp,data_hora_venda,tipo_transacao
2,3,PORTAS E JANELAS 3,ACRÍLICA SOBRE TELA,53 X 73,2024,"3.860,00","3.667,00",5% À VISTA/ 6 VEZES,TELAS,REF 03...PORTAS E JANELAS 3.jpg,disponível,link_pix,link_mp,data_hora_venda,tipo_transacao
3,4,PORTAS E JANELAS 4,ACRÍLICA SOBRE TELA,50 X 70,2024,"3.500,00","3.325,00",5% À VISTA/ 6 VEZES,TELAS,REF 04 ...PORTAS E JANELAS 4.jpg,disponível,link_pix,link_mp,data_hora_venda,tipo_transacao
4,5,PORTAS E JANELAS 5,ACRÍLICA SOBRE TELA,70 X 100,2024,"7.000,00","6.650,00",5% À VISTA/ 6 VEZES,TELAS,REF 05 - PORTAS E JANELAS 5.jpg,disponível,link_pix,link_mp,data_hora_venda,tipo_transacao


In [ ]:
# Gerar links de pagamento para cada obra disponível
print("Gerando links de pagamento...")

for i, row in df.iterrows():
    item_id = int(row.get('item', i))
    status_atual = row.get('status', 'disponível')
    
    # Verificar se a obra está disponível antes de gerar links
    if status_atual == 'disponível':
        titulo = row.get('telas', f'Obra {item_id}')
        preco = row.get('valor (r$)', 0)
        preco_pix = row.get('valor_pix', 0)
        
        # Gerar link de pagamento no crédito
        link_credito = criar_link_pagamento_credito(
            titulo=titulo,
            preco=preco,
            item_id=item_id
        )
        
        # Gerar link de pagamento PIX
        link_pix = criar_link_pagamento_pix(
            titulo=titulo,
            preco_pix=preco_pix,
            item_id=item_id
        )
        
        # Atualizar DataFrame com os links gerados
        if link_credito:
            df.at[i, 'link_mp'] = link_credito
            print(f"✓ Link crédito gerado para: {titulo}")
        else:
            df.at[i, 'link_mp'] = ''
            print(f"✗ Erro ao gerar link crédito para: {titulo}")
        
        if link_pix:
            df.at[i, 'link_pix'] = link_pix
            print(f"✓ Link PIX gerado para: {titulo}")
        else:
            df.at[i, 'link_pix'] = ''
            print(f"✗ Erro ao gerar link PIX para: {titulo}")
    else:
        # Obra não disponível - manter campos vazios ou com links desativados
        df.at[i, 'link_mp'] = '#indisponivel'
        df.at[i, 'link_pix'] = '#indisponivel'
        print(f"⊘ Obra {titulo} não disponível - links não gerados")

print("\n✅ Processo de geração de links concluído!")

Gerando links de pagamento...
Erro ao criar link de crédito para PORTAS E JANELAS 1: 'init_point'
Erro ao criar link de crédito para PORTAS E JANELAS 1: 'init_point'
Erro ao criar link PIX para PORTAS E JANELAS 1: 'init_point'
✗ Erro ao gerar link crédito para: PORTAS E JANELAS 1
✗ Erro ao gerar link PIX para: PORTAS E JANELAS 1
Erro ao criar link PIX para PORTAS E JANELAS 1: 'init_point'
✗ Erro ao gerar link crédito para: PORTAS E JANELAS 1
✗ Erro ao gerar link PIX para: PORTAS E JANELAS 1
Erro ao criar link de crédito para PORTAS E JANELAS 2: 'init_point'
Erro ao criar link de crédito para PORTAS E JANELAS 2: 'init_point'
Erro ao criar link PIX para PORTAS E JANELAS 2: 'init_point'
✗ Erro ao gerar link crédito para: PORTAS E JANELAS 2
✗ Erro ao gerar link PIX para: PORTAS E JANELAS 2
Erro ao criar link PIX para PORTAS E JANELAS 2: 'init_point'
✗ Erro ao gerar link crédito para: PORTAS E JANELAS 2
✗ Erro ao gerar link PIX para: PORTAS E JANELAS 2
Erro ao criar link de crédito para POR

In [ ]:
html_template_caminho = os.path.join(DIRETORIO_BASE, "HTML", "index.html")
with open(html_template_caminho, 'r', encoding='utf-8') as file:
    html_template = file.read()
print("Template HTML carregado com sucesso!")

Template HTML carregado com sucesso!


In [ ]:
for i, row in df.iterrows():
    idx_val = int(row.get('item', i))
    pasta = f"item_{idx_val}"
    caminho_pasta = os.path.join(DIRETORIO_SAIDA, pasta)
    os.makedirs(caminho_pasta, exist_ok=True)
    link_final_da_obra = f"{URL_BASE_QR}/qrcodes/{pasta}/index.html"
    df.at[i, 'qr link'] = link_final_da_obra

    titulo_encoded = urllib.parse.quote(str(row.get('telas', f'obra_{idx_val}')))
    
    # Obter status e links da obra
    status = row.get('status', 'disponível')
    link_mp = row.get('link_mp', '#')
    link_pix = row.get('link_pix', '#')
    
    # Configurar variáveis condicionais baseadas no status
    if status == 'vendido' or link_mp in ['#indisponivel', '#vendido'] or link_pix in ['#indisponivel', '#vendido']:
        classe_indisponivel = 'obra-indisponivel'
        classe_btn_disabled = 'btn-disabled'
        onclick_disabled = 'onclick="return false;"'
        aviso_vendido = '<div class="aviso-vendido">⚠️ OBRA INDISPONÍVEL - JÁ FOI VENDIDA</div>'
        link_pagamento_final = '#'
        link_pix_final = '#'
    else:
        classe_indisponivel = ''
        classe_btn_disabled = ''
        onclick_disabled = ''
        aviso_vendido = ''
        link_pagamento_final = link_mp
        link_pix_final = link_pix
    
    tecnica_ano = f"{row.get('técnica', '')}, {row.get('ano', '')}"

    conteudo_html = html_template.format(
        titulo=row.get('telas', ''),
        tecnica=tecnica_ano,
        dimensoes=row.get('dimensão', ''),
        preco=row.get('valor (r$)', ''),
        valor_pix=row.get('valor_pix', ''),
        link_pagamento=link_pagamento_final,
        link_pix_url=link_pix_final,
        titulo_encoded=titulo_encoded,
        imagem=row.get('fotos', ''),
        classe_indisponivel=classe_indisponivel,
        classe_btn_disabled=classe_btn_disabled,
        onclick_disabled=onclick_disabled,
        aviso_vendido=aviso_vendido
    )

    caminho_html = os.path.join(caminho_pasta, "index.html")
    with open(caminho_html, "w", encoding="utf-8") as arquivo_html:
        arquivo_html.write(conteudo_html)

    # Gerar QR Code
    qr = qrcode.QRCode(
        version=1,
        error_correction=qrcode.constants.ERROR_CORRECT_L,
        box_size=10,
        border=4,
    )
    qr.add_data(link_final_da_obra)
    qr.make(fit=True)

    imagem_qr = qr.make_image(fill_color="black", back_color="white")
    
    # Salvar QR code no diretório da obra (mantém compatibilidade)
    caminho_qr = os.path.join(caminho_pasta, "qrcode.png")
    imagem_qr.save(caminho_qr)
    
    # NOVO: Exportar QR code para pasta separada com nome simplificado
    qr_export_nome = f"qr_{idx_val}.png"
    qr_export_path = os.path.join(QR_EXPORT_DIR, qr_export_nome)
    imagem_qr.save(qr_export_path)
    
    df.at[i, 'qr_export_file'] = qr_export_nome

print("Páginas HTML e QR Codes gerados com sucesso!")
print(f"QR Codes exportados para: {QR_EXPORT_DIR}")

Páginas HTML e QR Codes gerados com sucesso!
QR Codes exportados para: c:\Users\pedro\Documents\qrCodeTelasExposicao\qr_codes_export


In [ ]:
# Instalar SDK do Mercado Pago
!pip install mercadopago

In [ ]:
# Configuração do Mercado Pago
import mercadopago

# IMPORTANTE: Substitua pelo seu Access Token de teste
# Obtenha em: https://www.mercadopago.com.br/developers/panel/credentials
ACCESS_TOKEN = 'APP_USR-8c3372df-d79c-4de2-bdf6-1764b05431a4'

# Inicializar SDK do Mercado Pago
sdk = mercadopago.SDK(ACCESS_TOKEN)

print("SDK do Mercado Pago configurado com sucesso!")

SDK do Mercado Pago configurado com sucesso!


In [ ]:
def criar_link_pagamento_credito(titulo, preco, item_id, link_notificacao=None):
    """
    Cria um link de pagamento no crédito via Mercado Pago
    
    Args:
        titulo: Nome da obra
        preco: Valor em R$ (formato string ou float)
        item_id: ID único da obra
        link_notificacao: URL para receber notificações de pagamento (webhook)
    
    Returns:
        URL do link de pagamento ou None em caso de erro
    """
    try:
        # Converter preço para float se for string
        if isinstance(preco, str):
            preco = float(preco.replace('.', '').replace(',', '.'))
        
        # Criar preference com configurações para crédito
        preference_data = {
            "items": [
                {
                    "title": titulo,
                    "quantity": 1,
                    "unit_price": float(preco),
                    "currency_id": "BRL"
                }
            ],
            "payment_methods": {
                "excluded_payment_types": [
                    {"id": "ticket"},  # Exclui boleto
                ],
                "installments": 6  # Até 6 parcelas
            },
            "back_urls": {
                "success": f"{URL_BASE_QR}/qrcodes/item_{item_id}/index.html",
                "failure": f"{URL_BASE_QR}/qrcodes/item_{item_id}/index.html",
                "pending": f"{URL_BASE_QR}/qrcodes/item_{item_id}/index.html"
            },
            "auto_return": "approved",
            "external_reference": f"obra_{item_id}_credito",
            "statement_descriptor": "PAULO CANUTO ARTE"
        }
        
        # Adicionar URL de notificação se fornecida
        if link_notificacao:
            preference_data["notification_url"] = link_notificacao
        
        # Criar preference
        preference_response = sdk.preference().create(preference_data)
        preference = preference_response["response"]
        
        return preference["init_point"]
    
    except Exception as e:
        print(f"Erro ao criar link de crédito para {titulo}: {e}")
        return None


def criar_link_pagamento_pix(titulo, preco_pix, item_id, link_notificacao=None):
    """
    Cria um link de pagamento via PIX no Mercado Pago
    
    Args:
        titulo: Nome da obra
        preco_pix: Valor com desconto PIX em R$ (formato string ou float)
        item_id: ID único da obra
        link_notificacao: URL para receber notificações de pagamento (webhook)
    
    Returns:
        URL do link de pagamento PIX ou None em caso de erro
    """
    try:
        # Converter preço para float se for string
        if isinstance(preco_pix, str):
            preco_pix = float(preco_pix.replace('.', '').replace(',', '.'))
        
        # Criar preference configurada apenas para PIX
        preference_data = {
            "items": [
                {
                    "title": f"{titulo} - PIX (5% desconto)",
                    "quantity": 1,
                    "unit_price": float(preco_pix),
                    "currency_id": "BRL"
                }
            ],
            "payment_methods": {
                "excluded_payment_types": [
                    {"id": "credit_card"},  # Exclui cartão de crédito
                    {"id": "debit_card"},   # Exclui cartão de débito
                    {"id": "ticket"}        # Exclui boleto
                ],
                "excluded_payment_methods": []
            },
            "back_urls": {
                "success": f"{URL_BASE_QR}/qrcodes/item_{item_id}/index.html",
                "failure": f"{URL_BASE_QR}/qrcodes/item_{item_id}/index.html",
                "pending": f"{URL_BASE_QR}/qrcodes/item_{item_id}/index.html"
            },
            "auto_return": "approved",
            "external_reference": f"obra_{item_id}_pix",
            "statement_descriptor": "PAULO CANUTO ARTE"
        }
        
        # Adicionar URL de notificação se fornecida
        if link_notificacao:
            preference_data["notification_url"] = link_notificacao
        
        # Criar preference
        preference_response = sdk.preference().create(preference_data)
        preference = preference_response["response"]
        
        return preference["init_point"]
    
    except Exception as e:
        print(f"Erro ao criar link PIX para {titulo}: {e}")
        return None

print("Funções de criação de links de pagamento definidas!")

Funções de criação de links de pagamento definidas!


In [ ]:
df.to_csv("obras_com_links.csv", index=False, encoding="utf-8")

In [ ]:
# Gerar QR Code para o catálogo completo
catalogo_link = "https://datacanuto.github.io/galeria-digital-artes/catalog_mobile/index.html"

qr_catalogo = qrcode.QRCode(
    version=1,
    error_correction=qrcode.constants.ERROR_CORRECT_L,
    box_size=10,
    border=4,
)
qr_catalogo.add_data(catalogo_link)
qr_catalogo.make(fit=True)

imagem_qr_catalogo = qr_catalogo.make_image(fill_color="black", back_color="white")

# Salvar QR code do catálogo
qr_catalogo_path = os.path.join(QR_EXPORT_DIR, "qr_catalogo_completo.png")
imagem_qr_catalogo.save(qr_catalogo_path)

print(f"QR Code do catálogo gerado com sucesso!")
print(f"Salvo em: {qr_catalogo_path}")

QR Code do catálogo gerado com sucesso!
Salvo em: c:\Users\pedro\Documents\qrCodeTelasExposicao\qr_codes_export\qr_catalogo_completo.png
